In [ ]:
#| default_exp tc.simple

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# First TC model

Sample TC model with a trade inpact model that is linear with the size of the number of shares/contracts. Denominator is the median traling volume and the numberator is the trade size. This model has not been calibrated and should not be assumed correct.

In [ ]:
#| export
import pandas as pd
import numpy as np

In [ ]:
#| export
class simple:
    def __init__(self, data, param):
        """
        Initialize the simple class with data and parameters.
        
        Parameters:
        - data: A numpy array containing OHLCV (Open, High, Low, Close, Volume) data.
        - param: A dictionary containing parameters 'tc_slope' and 'tc_bias'.
        """
        self.ohlcv = data  # OHLCV data, assumes a specific format
        self.slope = param['tc_slope']  # Slope parameter for transaction cost calculation
        self.bias = param['tc_bias']    # Bias parameter for transaction cost calculation

    def TC(self, trade):
        """
        Calculate the transaction cost (TC) based on the given trade volume.
        
        Parameters:
        - trade: The trade volume.
        
        Returns:
        - A dictionary containing the calculated transaction cost.
        """
        V = self.ohlcv[:, 5]  # Extract the volume data from the OHLCV array
        p = self.ohlcv[:, 4]  # Extract the price data from the OHLCV array
        
        # The data is daily. Assume a 24-hour session.
        v = V / (24 * 60)  # Calculate the volume per minute
        vm = np.median(v[-40:])  # Calculate the median volume per minute over the last 40 periods
        dp = np.diff(p)  # Calculate the price differences
        ss = np.std(dp[-40:])  # Calculate the standard deviation of price differences over the last 40 periods
        
        out = {}
        # Calculate the transaction cost using the slope, standard deviation, trade volume, and bias
        out['TC'] = float(self.slope * (ss * np.abs(trade) / vm) + self.bias)
        
        return out


## Example

In [ ]:
#| eval: false

# Get data
from backtest_sample.data_interface import get_data
import_module = get_data()
df = import_module('BTC-USD').sim_data()

[*********************100%***********************]  1 of 1 completed

In [ ]:
#| eval: false
# TC in the begging
prm = {}
prm['tc_slope'] = 1e3
prm['tc_bias'] = 100
tc = simple(df.values[:100,:],prm)
print(tc.TC(1000))

{'TC': 813.5629449701777}
{'TC': 813.5629449701777}


In [ ]:
#| eval: false
# TC in the end
prm = {}
prm['tc_slope'] = 1e3
prm['tc_bias'] = 100
tc = simple(df.values[-100:,:],prm)
print(tc.TC(300))

{'TC': 125.51845817251021}
{'TC': 125.51845817251021}
